# Group Details

## Group Name:

### Student 1:

### Student 2:

### Student 3:

# Loading Data and Preliminaries

In [9]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [10]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [11]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""

example = load_array('data/task 1/train/trajectory_0.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -5.159721083543527


In [12]:

example = load_array('data/task 1/train/trajectory_1.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

shape of initial state (model input): (5, 5)
shape of terminal state (to be predicted by model): (5, 2)


In [13]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time. 
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.

"""

example = load_array('data/task 2_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_array('data/task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-1.11611543  3.21149953         nan         nan]
 [-0.2865083   4.30801877         nan         nan]
 [ 1.07701594 -8.12529269         nan         nan]
 [-0.92053478  3.13709551         nan         nan]
 [-3.96308297 -4.27733589         nan         nan]
 [ 2.33945401 -8.67733599         nan         nan]
 [-4.83949085  3.67854952         nan         nan]
 [ 0.31080159 -9.74720071         nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import os

In [72]:
#Read data from multiple files
initial_state_train=[]
terminal_state_train=[]
for i in range(900):
    data =load_array('data/task 1/train/trajectory_'+str(i)+'.npz', task='task 1')
    initial_state_train.append(data[0])
    terminal_state_train.append(data[1])



In [73]:
initial_state_train[0]

array([[ 4.54673709, -5.27118739,  5.07863417, -1.09690628, -3.66929964],
       [ 3.57509525,  4.3270607 , -0.08095022, -0.57868726, -2.95971243],
       [ 2.67733735, -5.15972108,  5.35238208,  2.42652043,  1.45870728],
       [ 2.27455417, -6.79584511,  2.29632123,  0.30418238, -0.4065998 ],
       [ 2.38359341, -2.75861066,  1.77940931, -0.07643627, -0.35954359],
       [ 4.28596268,  0.34981219,  4.8286224 ,  1.35630962, -2.55760522],
       [ 3.03118516, -0.50562258, 15.04631712, -0.94772523, 14.98127867],
       [ 4.13530017,  1.81342682,  3.84375499, -0.6989711 , -2.25092411]])

In [ ]:
#For each intial state, measure the relative distance to the other bodies


In [16]:
#find the max length of the data
""" max_length=0
for i in range(900):
    if initial_state_train[i].shape[0]>max_length:
        max_length=initial_state_train[i].shape[0] """

' max_length=0\nfor i in range(900):\n    if initial_state_train[i].shape[0]>max_length:\n        max_length=initial_state_train[i].shape[0] '

# Data Handling and Preprocessing

In [17]:
#build a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.file_list = os.listdir(data_dir)

    def __getitem__(self, index):
        file_path = os.path.join(self.data_dir, self.file_list[index])
        # Read and preprocess the data from the file
        data = self._read_file(file_path)
        # Return the preprocessed data
        return data

    def __len__(self):
        return len(self.file_list)

    def _read_file(self, file_path):
        data= load_array(file_path, task='task 1')
        # Pad the data with zeros so that all sequences have length 9
        input_tensor= np.pad(data[0], ((0,9-len(data[0])),(0,0)), 'constant', constant_values=(0))
        target_tensor= np.pad(data[1], ((0,9-len(data[0])),(0,0)), 'constant', constant_values=(0))


        input_tensor = torch.tensor(input_tensor)
        target_tensor = torch.tensor(target_tensor)
        return input_tensor, target_tensor 


In [18]:
data_dir = 'data/task 1/train'

In [19]:
dataset = CustomDataset(data_dir)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Model Implementation

In [46]:
class DeepSet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepSet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),

        )
        self.aggregator = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim[0]*output_dim[1])
        )
        self.output_dim = output_dim

    def forward(self, x):
        batch_size = x.shape[0]
        encoded = self.encoder(x)
        aggregated = torch.mean(encoded, dim=1)
        aggregated = self.aggregator(aggregated)
        output = self.decoder(aggregated)
        output = output.view(batch_size, self.output_dim[0], self.output_dim[1])
        return output



In [68]:
import torch
import torch.nn as nn

class DeepSetRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim,activation= nn.LeakyReLU()):
        super(DeepSetRNN, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            
        )
        self.aggregator = nn.GRU(hidden_dim, hidden_dim, batch_first=True) # Replace with GRU

        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, hidden_dim),
            activation,
            nn.Linear(hidden_dim, output_dim[0] * output_dim[1])
        )
        self.output_dim = output_dim

    def forward(self, x):
        batch_size = x.shape[0]
        encoded = self.encoder(x)
        _, aggregated = self.aggregator(encoded)  # Use the RNN aggregator
        aggregated = aggregated.squeeze(0)  # Remove the sequence dimension
        output = self.decoder(aggregated)
        output = output.view(batch_size, self.output_dim[0], self.output_dim[1])
        return output


In [69]:
input_dim = 5
hidden_dim = 64
output_dim = [9,2]  # Output x and y coordinates
batch_size = 32
num_epochs = 50

# Model Training

In [63]:
model = DeepSet(input_dim, hidden_dim, output_dim)

# Create the data loader
dataset = CustomDataset(data_dir)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [64]:
test_data_dir = 'data/task 1/test'
test_dataset = CustomDataset(test_data_dir)
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)



In [70]:
model = DeepSetRNN(input_dim, hidden_dim, output_dim)

# Create the data loader
dataset = CustomDataset(data_dir)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [71]:
for epoch in range(num_epochs):
    model.train()
    for input_tensor, target_tensor in data_loader:
        #set input and target to float
        input = input_tensor.to('cpu').float()
        target = target_tensor.to('cpu').float()

        # Forward pass
        output = model(input).to('cpu')
        # Compute the loss
        loss = criterion(output, target)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    test_loss=0.0
    with torch.no_grad():
        for input_tensor, target_tensor in test_data_loader:
            input = input_tensor.to('cpu').float()
            target = target_tensor.to('cpu').float()
            output = model(input).to('cpu')
            test_loss += criterion(output, target_tensor).item()
    test_loss /= len(test_data_loader)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}, Test Loss: {test_loss}")

Epoch [1/50], Loss: 12.599453926086426, Test Loss: 14.747041029559664
Epoch [2/50], Loss: 15.24623966217041, Test Loss: 14.212319937163151
Epoch [3/50], Loss: 9.712862014770508, Test Loss: 15.662670619772591
Epoch [4/50], Loss: 17.36570167541504, Test Loss: 14.065373895045274
Epoch [5/50], Loss: 12.51584243774414, Test Loss: 14.187860914964032
Epoch [6/50], Loss: 9.923476219177246, Test Loss: 12.975637932418723
Epoch [7/50], Loss: 9.939854621887207, Test Loss: 12.88765626277024
Epoch [8/50], Loss: 18.01913070678711, Test Loss: 12.952148250987916
Epoch [9/50], Loss: 13.629838943481445, Test Loss: 13.140239588249504
Epoch [10/50], Loss: 12.470887184143066, Test Loss: 13.359621736682953
Epoch [11/50], Loss: 17.428478240966797, Test Loss: 13.773844002575249
Epoch [12/50], Loss: 14.444316864013672, Test Loss: 12.438155121061193
Epoch [13/50], Loss: 12.469012260437012, Test Loss: 12.827209521209065
Epoch [14/50], Loss: 9.219258308410645, Test Loss: 11.945247133675291
Epoch [15/50], Loss: 14.

In [41]:
for epoch in range(num_epochs):
    for batch in data_loader:
        print(type(batch))
        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, batch)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

<class 'list'>


TypeError: linear(): argument 'input' (position 1) must be Tensor, not list

# Evaluation

In [ ]:
#todo